# Coletando os dados

O programa realizará a coleta de dados a partir de WebScraping com Selenium do site Zapimóveis. Será utilizado o driver do Google Chrome para esse processo.
Ao final, será gerado um arquivo .csv que armazenará todos os dados consultados.

## Bibliotecas

Algumas bibliotecas são necessárias para a realização do WebScraping...

In [1]:
!pip install selenium

In [9]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException

import pandas as pd

## WebScraping

Definindo as caracteristicas da janela Desktop.

In [86]:
## Definição de Janela Desktop
options = Options()
options.add_argument("--incognito")
options.add_argument("start-maximized")

Abrindo o browser e acessando a url definida.

In [87]:
driver = webdriver.Chrome(options=options)
url = f'https://www.zapimoveis.com.br/venda/apartamentos/sp+sao-paulo/?transacao=Venda&tipoUnidade=Residencial,Apartamento&tipo=Im%C3%B3vel%20usado&onde=,S%C3%A3o%20Paulo,S%C3%A3o%20Paulo,,,,,city,BR%3ESao%20Paulo%3ENULL%3ESao%20Paulo,-23.024996,-45.563879,%2Fvenda%2Fapartamentos%2Fsp%2Bsao-paulo%2F'
driver.get(url)
time.sleep(2)

Função criada para iterar via webscraping pelas páginas do site Zapimóveis.

In [88]:
def webscraping_zapimoveis(num_interation = 10):  
    
    list_property_title = []
    list_property_title = []
    list_price = []
    list_property_area = []
    list_property_room = []
    list_property_bath = []
    list_property_garage = []
    list_property_address = []
    
    success = 0
    
    for num in range(1, num_interation+1):
        try:
            list_pag = driver.find_element(By.XPATH, '//*[@id="app"]/section[1]/div[2]/div[3]/section')

            #Título da propriedade
            property_title = list_pag.find_elements(By.CSS_SELECTOR ,'.simple-card__text')
            list_property_title = list_property_title + [el.text for el in property_title]

            #Preço da propriedade
            price = list_pag.find_elements_by_css_selector('.simple-card__price')
            list_price = list_price + [el.text for el in price]

            #Área da propriedade
            property_area = list_pag.find_elements_by_css_selector('.js-areas')
            list_property_area = list_property_area + [el.text for el in property_area]

            #Quarto da propriedade
            property_room = list_pag.find_elements_by_css_selector('.js-bedrooms')
            list_property_room = list_property_room + [el.text for el in property_room]

            #Banheiro da propriedade
            property_bath = list_pag.find_elements_by_css_selector('.js-bathrooms')
            list_property_bath = list_property_bath + [el.text for el in property_bath]

            #Garagem da propriedade
            property_garage = list_pag.find_elements_by_css_selector('.js-parking-spaces')
            list_property_garage = list_property_garage + [el.text for el in property_garage]

            #Endereço da propriedade
            property_address = list_pag.find_elements_by_css_selector('.simple-card__address')
            list_property_address = list_property_address + [el.text for el in property_address]   

            #Click_buttom
            if num == 1:
                n = 6
            else:
                n = 7
            buttom_var = f'#app > section.results__container > div.results__wrapper > div.results__list.js-results > section > ul > li:nth-child({n}) > button'

            driver.execute_script("return arguments[0].scrollIntoView(true);", WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, buttom_var))))
            time.sleep(2)
            botao_proximo = list_pag.find_element(By.CSS_SELECTOR, buttom_var)
            botao_proximo.click()
            time.sleep(2)

            print(f"Page {num} done. Next page...")
            success += 1
        except:
            pass
        
    print(f"Process finished. {num_interation} successful iterations.")
    
    columns_name = ['property_describe', 'price', 'area', 'room', 'bath', 'garage', 'address']
    df = pd.DataFrame(zip(list_property_title, list_price, list_property_area, list_property_room, list_property_bath, list_property_garage, list_property_address), columns=columns_name)

    return df

Chamando a função webscraping_zapimoveis() definida acima.

In [89]:
webscraping_zapimoveis()

Page 1 done. Next page...
Page 2 done. Next page...
Page 3 done. Next page...
Page 4 done. Next page...
Page 5 done. Next page...
Page 6 done. Next page...
Page 7 done. Next page...
Page 8 done. Next page...
Page 9 done. Next page...
Process finished. 10 successful iterations.


,property_describe,price,area,room,bath,garage,address
0,Condomínio fechado com apartamentos de 2 dormi...,R$ 162.990,40 m²,2,1,1,"Rua Gaspar Vaz da Cunha, Caixa D Agua"
1,Lindos apartamentos na planta de 2 e 3 dormitó...,R$ 229.000,68 - 82 m²,2 - 3,,1 - 2,"Rua Padre Fischer, Parque São Luís"
2,Condomínio fechado com apartamentos de 2 dormi...,R$ 163.990,40 m²,2,1,1,"Avenida Arnaldo Felipe Sbruzzi, Piracangagua"
3,Maiores detalhes em contato pelo whatssapp,R$ 650.000,84 m²,3,2,1,"Rua Abagiba, Saúde"
4,Condomínio altíssimo padrão próximo ao shoppin...,R$ 3.000.000,240 m²,3,5,4,"Rua Tupi, Santa Cecília"
...,...,...,...,...,...,...,...
215,"Apartamento de 2 qtos premium, lazer completo ...",R$ 430.000,64 m²,3,1,1,"Avenida Celso Garcia, Tatuapé"
216,"Lindo apartamento com 425 m², bem conservado, ...",Sob consulta,42 - 83 m²,2,2,1,"Avenida Chiyo Yamamoto, Vila Nova Bonsucesso"
217,Em Construção- Entrega em Fevereiro de 2023. O...,R$ 255.000,44 - 46 m²,2,1,1,"Rua Barão de Jaceguai, Centro"
218,"Lindo apartamento 3 dormitórios , repleto de a...",Sob consulta,51 m²,2,6,1,"Avenida Sebastião de Paula, São João"


## Gerando arquivo .CSV

In [90]:
df.to_csv('C:/Users/rodol/Desktop/Awari/df_zapimoveis.csv', index=False, sep='|', encoding="utf-8-sig")